In [2]:
import pandas as pd

In [2]:
df_predictions = pd.read_csv("test_dataset__bart_model_language_rating_prediction.csv")

In [3]:
df = pd.read_csv("kidsInMindSubtitles2004.csv")
df.shape

(2004, 15)

In [6]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

/tmp/ipykernel_533189/2564618054.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


In [7]:
query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][0], 1))+" \
; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][0] + " Generate \
language summary like kids in mind website? Please consider the below two example language summaries of \
Please consider the below two example language summaries of \
movies:  \n 1." + df['Language_Description'][1] + "\n 2."+df['Language_Description'][1]

In [8]:
query

"Given the subtitle rating of a movie = 6.5 ; and a list of inappropriate words : ['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'goddamn', 'fat', 'shit', 'masturbate', 'hell', 'fuck', 'fat', 'fat', 'toilet', 'dead', 'skank', 'prick', 'ass', 'fuck', 'cock', 'toilet', 'toilet', 'toilet', 'fucking', 'tinkle', 'ugly', 'butt', 'stupid', 'stupid', 'hell', 'fucked', 'killing', 'fuck', 'sex', 'tits', 'bulldyke', 'damn', 'fat', 'stupid', 'ass', 'fucking', 'piss', 'desire', 'desire', 'desire', 'burn', 'tongue'] Generate language summary like kids in mind website? Please consider the below two example language summaries of Please consider the below two example language summaries of movies:  \n 1. About 67 F-words and its derivatives, 2 obscene hand gestures, 12 sexual references, 38 scatological terms, 38 anatomical terms, 39 mild obscenities, name-calling (crazy, moron, idiots, dumb, sluts, stupid, psycho, punk, Negro, drunks, school boy, boring, pussy whipped, mini-mite, corny, Little Miss Overac

In [10]:
query = """ Given the subtitle rating of a movie = 6.5 ; and a list of inappropriate words : 
            'bondage', 'bitch', 'ass', 'fuck', 'fuck', 'goddamn', 'fat', 'shit', 'masturbate', 
            'hell', 'fuck', 'fat', 'fat', 'toilet', 'dead', 'skank', 'prick', 'ass', 'fuck', 'cock', 
            'toilet', 'toilet', 'toilet', 'fucking', 'tinkle', 'ugly', 'butt', 'stupid', 'stupid', 
            'hell', 'fucked', 'killing', 'fuck', 'sex', 'tits', 'bulldyke', 'damn', 'fat', 'stupid', 
            'ass', 'fucking', 'piss', 'desire', 'desire', 'desire', 'burn', 'tongue'"""

In [25]:


import os
import openai

os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"


openai.organization = ""

openai.api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"



#openai.Model.list()



In [26]:


response_id_done = {}
import re
import json

import openai  # for OpenAI API calls
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [27]:

from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"


client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="My API Key",
)
from docx import Document

def transcribe_audio(audio_file_path):
    with open(audio_file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create("whisper-1", audio_file)
    return transcription['text']

In [28]:
model_name = 'gpt-4'

In [29]:
response = client.chat.completions.create( model="gpt-4",
                                          temperature=0,
                                          messages=[{
                                              "role": "system",
                                              "content": query, 
                                          }
                                          ]
                                         )

In [30]:
print(response.dict()['choices'][0]['message']['content'])

The movie contains about 11 instances of the F-word and its derivatives, 3 sexual references, 4 scatological terms (toilet), 5 anatomical terms (ass, tits), 6 mild obscenities (hell, damn), name-calling (fat, ugly, stupid, skank, prick), and 3 instances of desire which could be interpreted as a sexual reference. There are also references to bondage and masturbation, which are sexual in nature. There are no obscene hand gestures, stereotypical references, religious obscenities, or religious exclamations noted. The subtitle rating of the movie is 6.5, indicating a moderate level of inappropriate language.


/tmp/ipykernel_509292/2686690461.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  print(response.dict()['choices'][0]['message']['content'])


In [35]:
#df_predictions['Predcited_Score'][0]

In [45]:
df_predictions['gpt_4_response'] = None

In [53]:

for idx in tqdm(range(len(df_predictions))):
    
    #idx += 24
    
    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" \
    ; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]
    
    response = client.chat.completions.create( model="gpt-4",
                                          temperature=0,
                                          messages=[{
                                              "role": "system",
                                              "content": query, 
                                          }
                                          ]
                                         )
    
    with open("gpt_4_reponse_v1.json", "a") as file:
        file.write(json.dumps(str(idx)+":"+str(response)+"\n"))
        
    df_predictions['gpt_4_response'][idx] = response.dict()['choices'][0]['message']['content']
    

  0%|          | 0/201 [00:00<?, ?it/s]

/tmp/ipykernel_509292/3526773210.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  df_predictions['gpt_4_response'][idx] = response.dict()['choices'][0]['message']['content']
/tmp/ipykernel_509292/3526773210.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['gpt_4_response'][idx] = response.dict()['choices'][0]['message']['content']
/tmp/ipykernel_509292/3526773210.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  df_predictions['gpt_4_respons

KeyError: 201

In [3]:
df_predictions = pd.read_csv("df_predictions_gpt_4_response.csv")

In [4]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [5]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['language_summary'][idx], df_predictions['gpt_4_response'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [6]:
sum(rougeL_scores)/len(rougeL_scores)

0.13890713869756618

In [7]:
df_predictions.head(2)

,Language_Score,Predcited_Score,subtitle,language_summary,cleaned_text,inappropriate_words,inappropriate_words_count,gpt_4_response
0,6,6.543292,The High Council\nhas reached a verdict --guil...,"13 F-words, 1 nearly uttered F-word, 3 obscene...",the high council reached verdict --guilty!you ...,"['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'g...",47,"\""The movie contains a significant amount of s..."
1,7,7.480484,"Danny, give me my shirt!Come and get it.Don't ...","About 25 F-words, 2 obscene hand gestures, 8 s...","danny, shirt!come it.don't think maybe we're t...","['douche', 'damn', 'bitch', 'bra', 'sex', 'lai...",103,"\""The movie contains a high frequency of stron..."


In [8]:
import bert_score

In [9]:
references = list(df_predictions['language_summary'])
summaries = list(df_predictions['gpt_4_response'])

In [10]:
P, R, F1 = bert_score.score(summaries, references, lang='en')
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.8223161697387695
BERTScore Recall: 0.8658444285392761
BERTScore F1: 0.8429979085922241
